In [2]:
# get list of systems

import json
import boto3

bucket="busobservatory-migration"
region="us-east-1"
config_object_key = "_bus_observatory_config.json"
    
s3 = boto3.resource("s3")
obj = s3.Object(bucket, config_object_key)

config = json.load(obj.get()['Body'])

In [ ]:
# iterate over systems

for system_id, system_config in config.items():
    
    # 
    
    